In [48]:
%cd ..
!ls

/exp/exp1/acp21rjf/deliberation/speachy
ami		    model_utils.py	   tedlium
checkpoints	    mwer_rescoring.py	   test_66_rescored.pkl
checkpoints_done    non_iid_dataloader.py  tools.py
experiment_configs  pg191kwft.json	   top100_5kb_history_250.pkl
hyps		    pg19mwerrr.json	   train_LM.py
ipynbs		    __pycache__		   train_lm.sh
IS_30s_utt.json     README.md		   train_mwer_lm.sh
IS_single_utt	    setup.py		   train_no_noise_10.pkl
lm		    speachy		   wandb
lm_utils.py	    speachy.egg-info	   wip
log.txt		    sweep_configs
model_configs	    sweep.yaml


In [49]:
from Levenshtein import distance

In [50]:
import pickle as pkl, numpy as np
from speachy.rescoring.tools import (
        sort_hypothesis_by_recording, 
        order_recordings_by_start_time,
        interpolate
)
from functools import partial, reduce
import torch

[NeMo W 2023-01-27 11:24:39 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-27 11:24:39 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [148]:
with open('train_do_40.pkl', 'rb') as f:
    test_beams = pkl.load(f)

In [149]:
test_beams = order_recordings_by_start_time(sort_hypothesis_by_recording(test_beams))

In [207]:
def get_edit_distance(hyps, target):
    return list(map(lambda x: distance(x, target) / len(target), hyps))
    
def create_samples_from_recording(recording, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    prev_end = None
    if shuffle == False:
        np.random.seed(42) # deterministic selection of negatives
    for i, utterance in enumerate(recording):
        start_t, end_t = utterance['meta_data']['timings'].values()
        if prev_end is None or (start_t - prev_end) > max_gap:
            samples.append([]) 
        if len(samples[-1]) >= num_utterances:
            samples.append([])
        hyps = utterance['beams'][0]
        hyps = list(map(lambda x: x['text'], list(hyps.values())))
        target = utterance['targets'][0]
        hyps = list(filter(lambda el:el != target, hyps))
        hyps = np.random.choice(hyps, min(num_negatives, len(hyps)), replace=False).tolist()
        examples = [target] + hyps
        error_rates = get_edit_distance(examples, target)
        samples[-1].append((examples, error_rates))
        prev_end = end_t
        
    return samples

In [151]:
def create_dataset_samples(recordings, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    for recording in recordings.keys():
        samples += create_samples_from_recording(recordings[recording], num_utterances, num_negatives, max_gap, shuffle)
    if shuffle:
        np.random.shuffle(samples) # shuffle samples
    return samples

In [152]:
def tokenize_and_pad(utterances, tokenizer): # returns padded utterances and their lengths
    tokenized = [tokenizer.text_to_ids(utt) for utt in utterances]
    max_len = max(map(len, tokenized))
    padded_utts = np.array([utt + [0] * (max_len - len(utt)) for utt in tokenized])
    return padded_utts, np.array(list(map(len, tokenized)))

In [56]:
flatten_nested_list = lambda l: [item for sublist in l for item in sublist]

In [90]:
def get_sub_batches(batch, tokenizer):
    proc_utts = lambda utts: tokenize_and_pad(flatten_nested_list(utts), tokenizer)
    sub_batches = []
    max_len = max(map(len, batch))
    for i in range(max_len):
        sub_batches.append({'utterances': [],'scores': [],'sb_lengths': [],})
        for el in batch:
            case = len(el) > i
            sub_batches[-1]['utterances'].append(el[i][0] if case else -1)
            sub_batches[-1]['scores'].append(el[i][1] if case else -1)
            sub_batches[-1]['sb_lengths'].append(len(el[i][0]) if case else -1)
     
    non_empty_indices = np.arange(len(sub_batches[0]['sb_lengths']))

    for i, sub_batch in enumerate(sub_batches):
        sb_utts, sb_scores, sb_lengths = [np.array(el, dtype=object) for el in (sub_batch['utterances'], sub_batch['scores'], sub_batch['sb_lengths'])]
        non_empty = non_empty_indices[sb_lengths != -1]
        # slice based on non empty of previous sub batch
        prev_fetch = None
        if i != 0:
            prev_lengths = sub_batches[i-1]['sb_lengths']
            prev_non_empty = sub_batches[i-1]['non_empty']
            diff_from = ((prev_lengths != -1) == (sb_lengths != -1))[prev_non_empty]
            prev_fetch = np.arange(len(prev_non_empty))[diff_from]
            
        sub_batches[i] = {
            'utterances': sb_utts,
            'scores': sb_scores,
            'sb_lengths': sb_lengths,
            'non_empty': non_empty,
            'prev_fetch': prev_fetch 
        }
    
    for i, sub_batch in enumerate(sub_batches):
        padded_utts, acc_lengths = proc_utts(sub_batch['utterances'][sub_batch['non_empty']].tolist())
        text_utts = sub_batch['utterances'][sub_batch['non_empty']].tolist()
        sub_batches[i] = {
            'utterances': torch.as_tensor(padded_utts),
            'lengths': torch.as_tensor(acc_lengths),
            'text_utts': flatten_nested_list(text_utts),
            'scores': torch.tensor(flatten_nested_list((sub_batch['scores'][sub_batch['non_empty']]).tolist())),
            'sb_lengths': torch.as_tensor(sub_batch['sb_lengths'][sub_batch['non_empty']].astype(int)),
            'prev_fetch': torch.as_tensor(sub_batch['prev_fetch']) if sub_batch['prev_fetch'].__class__.__name__ != 'NoneType' else None# indices to fetch the states from previous sub batch
        }
    return sub_batches

In [58]:
a = None
a.__class__.__name__

'NoneType'

In [59]:
class Sampler(object):
    def __init__(self, samples, batch_size, tokenizer, shuffle=True):
        self.samples = samples
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.shuffle = shuffle
        self.sample_indices = np.arange(len(self.samples))
        np.random.shuffle(self.sample_indices) if self.shuffle else None
        self.generator = self.__generator__()

    def __generator__(self):
        for i in range(0, len(self.samples), self.batch_size):
            yield get_sub_batches([self.samples[i] for i in self.sample_indices[i:i+self.batch_size]], self.tokenizer)

    def __iter__(self):
        return self

    def __list__(self):
        return list(self.generator)

    def __len__(self):
        return len(self.samples)

    def __next__(self):
        return next(self.generator)



def sampler(samples, batch_size, tokenizer, shuffle=True):
    sample_indices = np.arange(len(samples))
    np.random.shuffle(sample_indices) if shuffle else None
    batches = []
    for i in range(0, len(samples), batch_size):
        yield get_sub_batches([samples[i] for i in sample_indices[i:i+batch_size]], tokenizer)
        

In [60]:
import tools

In [61]:
tk = tools.load_tokenizer('./tedlium/tokenizers/tokenizer_spe_bpe_v1000/tokenizer.model')

In [62]:
samples[0][-1][0]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/3150807623.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/3150807623.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'samples' is not defined

In [63]:
np.array([['f','z'],['helo']]).tolist()

[NeMo W 2023-01-27 11:24:45 nemo_logging:349] /tmp/ipykernel_1906092/3555533298.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      np.array([['f','z'],['helo']]).tolist()
    


[['f', 'z'], ['helo']]

In [64]:
s1[-20]['prev_fetch']

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/2132013132.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/2132013132.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's1' is not defined

In [65]:
s1[0].keys()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/139049613.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/139049613.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's1' is not defined

In [66]:
s1[-1]['scores']

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/2845055575.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/2845055575.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's1' is not defined

In [67]:
torch.as_tensor(s1[-1]['lengths'].astype(int)).dtype

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/1701062046.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/1701062046.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's1' is not defined

In [68]:
loader.beep = '0'

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/2766218295.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/2766218295.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'loader' is not defined

In [187]:
loader = Sampler(samples, 3, tk, shuffle=False)
s1 = next(loader)

In [107]:
samples[0][6]

(["if it works as advertised then it's just clear as day because the economics the energy density are so different than nuclear as we know it"],
 [0.0])

In [115]:
s1[0].keys()

dict_keys(['utterances', 'lengths', 'text_utts', 'scores', 'sb_lengths', 'prev_fetch'])

In [169]:
s1 = next(loader)

In [197]:
s1[0]['sb_lengths']

tensor([6, 6, 6])

In [238]:
smaxed_scores = []
wer_scores = []
sb_lengths = s1[0]['sb_lengths']
sb_scores = -s1[0]['scores']
sb_starts = sb_lengths.cumsum(dim=0) - sb_lengths
sb_ends = sb_starts + sb_lengths
for i in range(len(sb_lengths)):
    smaxed_scores.append(wer_scores[sb_starts[i]:sb_ends[i]].mean() )
    

#smaxed_scores = torch.cat(smaxed_scores)
smaxed_scores[-1].mean()

tensor(0.1667)

In [217]:
1444 // 26

55

In [215]:
torch.randn(1444).view(27, 43)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/3564341899.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/3564341899.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: shape '[27, 43]' is invalid for input of size 1444

In [194]:
(s1[0]['utterances']).shape

torch.Size([18, 66])

In [172]:
s1[8]['text_utts']

["now social organization has reached the global level and i guess if there's good news i can say i'm bringing you it's just that all the salvation of the world requires is the intelligent",
 "now social organization has reached the global level and i guess the good news i say irving you it's just that all the sound of the world requires is the intelligent",
 "now social organization has reached the global level and i guess the good news i say irving you it's just that all the event of the world requires is intelligen",
 "now social organization has reached the global level and i guess the good news i say irving you it's just that all the sound of the world requires is the intelligen",
 "now social organization has reached the global level and i guess the good news i can say irving you it's just that all the event of the world requires is the intelligen",
 "now social organization has reached the global level and i guess the good news i can say irving you it's just that all the event o

In [116]:
N = 0
print(s1[1]['prev_fetch'])
print(s1[0]['text_utts'])

tensor([0, 1, 2])
["which was delicious and i don't like fish skin i don't like it seared i don't like it crispy", "which was delicious and i don't like fish skin i don't like its seirred i don't like it crisspyk", "which was delicious and i don't like fish skin i don't like it' sered i don't like it chris bees", "which was delicious and i don't like fish skin i don't like its seirred i don't like it chriss beake", "which was delicious and i don't like fish skin i don't like its seird i don't like it chriss beech", "which was delicious and i don't like fish skin i don't like it' sered i don't like it chris bea", 'this second fish', 'u this second fish', 'up this second fish', 'upp this second fish', 'u the second fish', 'p this second fish', "so i've known a lot of fish in my life", "itest so i've know a lot of fish in my life", "yes so i've known a lot of fish in my life", "yjusest so i' have known a lot of fish in my life", "jusht so i' have known a lot of fish in my life", "itesht s

In [119]:
s1 = next(loader)

In [72]:
loader.batch_size

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/3592700816.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/3592700816.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'loader' is not defined

In [73]:
z[0]['utterances'].shape

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/3858181404.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/3858181404.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'z' is not defined

In [74]:
for i in loader:
    z=i
    break

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/4283890645.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/4283890645.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'loader' is not defined

In [75]:
s1 = next(loader)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/4215673618.py:1 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/4215673618.py'                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'loader' is not defined

In [77]:
loader = sampler(samples, 5, tk)
s1 = next(loader)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_1906092/154326804.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_1906092/154326804.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'samples' is not defined

In [153]:
samples = create_dataset_samples(test_beams, num_negatives=5, num_utterances=100)

In [105]:
hyps

array(["i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars handy never a' writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my theisarres handy never im writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for italian wired i always keep my the sars handy never are and writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy never ionm writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sarres handy i never a writing anything but"],
      dtype='<U188')

In [106]:
hyps = list(map(lambda el:el['text'],list(test_beams['AimeeMullins_2009P'][0]['beams'][0].values())))
target = test_beams['AimeeMullins_2009P'][0]['targets'][0]
hyps = list(filter(lambda el:el != target, hyps))
# select 5 random hyps
hyps = np.random.choice(hyps, min(5, len(hyps)), replace=False).tolist()
hyps = hyps + [target]

In [97]:
import torch

In [123]:
list(zip(get_edit_distance(hyps, target), hyps))

[(0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f sars handy i've never writing anything but"),
 (0.0903954802259887,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the f sars handy i never writing anything but"),
 (0.1016949152542373,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy i never i a and writing anything but"),
 (0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my th the sars handy i never writing anything but"),
 (0.06779661016949153,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars han

In [117]:
(torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor(0.0753)

In [115]:
(torch.tensor(get_edit_distance(hyps, target))) - (torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor([ 0.0207,  0.0151,  0.0264,  0.0207, -0.0075, -0.0753])

In [1]:
import torch

In [5]:
sb_lengths = torch.tensor([6,6,4,6])
sb_indices = torch.arange(sb_lengths.size(0))
sb_lengths
sb_indices

tensor([0, 1, 2, 3])

In [18]:
torch.cat([sb_indices[ix].repeat(sb_lengths[ix]) for ix in range(sb_lengths.size(0))])

tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3])

In [14]:
sb_lengths[None].T

tensor([[6],
        [6],
        [4],
        [6]])

In [36]:
B*N

240

In [181]:
B, N, TD = 40, 60, 10
lgts = torch.randn(B, N, TD)
tgts = torch.randint(0, TD, (B, N))
ce1 = torch.nn.functional.cross_entropy(lgts.view(-1, lgts.size(-1)), tgts.view(-1), ignore_index=0, reduction='none').view(B, N)

print(ce1.shape)

torch.Size([40, 60])


In [184]:
ce1.mean()

tensor(2.4479)

In [46]:
tgts[0]

tensor([0, 5, 8, 0, 6, 4, 3, 4, 1, 9, 8, 1, 7, 3, 2, 2, 4, 4, 6, 4, 0, 3, 4, 2,
        8, 4, 1, 1, 0, 6, 5, 9, 7, 1, 2, 1, 5, 7, 5, 8, 0, 4, 5, 0, 3, 1, 1, 8,
        9, 7, 1, 9, 2, 0, 2, 2, 2, 7, 0, 6])

In [41]:
ce1*2

tensor([3.2967, 0.0000, 2.8007,  ..., 7.2741, 5.8558, 7.4246])

In [222]:
(ce1).squeeze()[:,None].shape

torch.Size([40, 1, 60])